In [53]:
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd

import nest_asyncio
nest_asyncio.apply()

In [54]:
import aiohttp
import asyncio
import time

start_time = time.time()

curr_season = 121
hs_mode = 'wild'


async def get_page_players(session, url):
    async with session.get(url) as resp:
        pagejson = await resp.json()
        rows = pagejson['leaderboard']['rows']
        return rows


async def getCurrentLeaderboardAsync():


    url = f'https://hearthstone.blizzard.com/en-us/api/community/leaderboardsData?region=US&leaderboardId={hs_mode}&page=1&seasonId={curr_season}'
    html = requests.get(url)
    text = html.text
    son = json.loads(text)
    totalPages = son['leaderboard']['pagination']['totalPages']

    async with aiohttp.ClientSession() as session:
        df = pd.DataFrame()
        tasks = []
        for page in range(1, totalPages+1):
            url = f'https://hearthstone.blizzard.com/en-us/api/community/leaderboardsData?region=US&leaderboardId={hs_mode}&page={page}&seasonId={curr_season}'
            tasks.append(asyncio.ensure_future(get_page_players(session, url)))

        allPages = []

        jsonObjects = await asyncio.gather(*tasks)
        for jsonFile in jsonObjects:
            allPages.append(jsonFile)

            rankList = []
            nameList = []

            # iterate for every page
            for page in allPages:

                for player in page:

                    name = player['accountid']
                    rank = player['rank']
                    rankList.append(rank)
                    nameList.append(name)
    df['rank'] = rankList
    df['name'] = nameList
    df = df.set_index('rank')
    return df
        

In [66]:
def findRank(playerName, df):
    try: return int(df.loc[df['name'] == playerName].index[0])
    except: 
        print(f'Player {playerName} not found in Legend rank, make sure you spelled name correctly')
        return False

def hasStarBonus(playerName, df):
    
    star11 = findRank(playerName,df) < (df.shape[0] / 10) 
    return str(star11)



In [56]:
# run this to import csv 

def importCSVLeaderboard(fileName):

    df_imported = pd.read_csv(fileName)
    df_imported = df_imported.set_index('rank')
    df_imported.head(10)
    return df_imported

In [64]:
def playerStats(df,name):

    # If they are legend and therefore ARE in the Database
    if findRank(name,df):

        print('Total Players Legend: ' + str(len(df.index)))
        print(f'Rank of {name}: ' + str(findRank(name,df)))
        print(f'Does {name} have a Star Bonus?: ' + str(hasStarBonus(name,df)))

    else:
        #print(f'Player {name} not found in Wild Legend.')
        pass

In [58]:
# see how many duplicate names in the dataframe
def findDuplicateNames(df):

    namesList = []
    duplicateNames = []
    legendNames = list(df['name'])
    for name in legendNames:
        if name in namesList:
            duplicateNames.append(name)
        
        namesList.append(name)

    return duplicateNames


def renameDuplicates(df):

    namesList = []
    legendNames = list(df['name'])
    for name in legendNames:
        if name in namesList:
            name = name + '*'
        
        namesList.append(name)

    df['name'] = namesList

    if len(findDuplicateNames(df)) != 0:
        return renameDuplicates(df)
    else: return df


In [59]:
from datetime import datetime
def timeStamp():
    # datetime object containing current date and time
    now = datetime.now()
    dt_string = now.strftime("%d.%m.%Y.%H.%M.%S")
    return dt_string

In [60]:
def storeCurrentLeaderboard():

    try:

        df = asyncio.run(getCurrentLeaderboardAsync())
        df = renameDuplicates(df)
        df.to_csv('WildLeaderboard.' + timeStamp())

    except:
        print('Error: Couldnt Store Leaderboard')
        return

    return df



In [61]:
recently_stored = storeCurrentLeaderboard()
recently_stored.head()


,name
rank,
1,ShuBIAO
2,JudusRising
3,BaoXUeSB
4,vdae
5,Saratoga0v0


In [67]:
#importCSVLeaderboard('WildLeaderboard.29.11.2022.11.30.38')
playerStats(recently_stored,'Abstraction')
playerStats(recently_stored,'Nimrod20')

Player Abstraction not found in Legend rank, make sure you spelled name correctly
Player Nimrod20 not found in Legend rank, make sure you spelled name correctly


In [63]:
# from apscheduler.schedulers.blocking import BlockingScheduler

# # store the current leaderboard once every 30 minutes while this is running
# scheduler = BlockingScheduler()
# scheduler.add_job(storeCurrentLeaderboard, 'interval', minutes=20)
# scheduler.start()